In [1]:
pip install uv


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
uv --version

uv 0.9.5
Note: you may need to restart the kernel to use updated packages.


In [3]:
uv init


error: Project is already initialized in `/workspaces/machine-learning-zoomcamp-homework/machine_learning_zoomcamp_homework_05` (`pyproject.toml` file exists)
Note: you may need to restart the kernel to use updated packages.


In [4]:
uv add scikit-learn==1.6.1

Resolved 6 packages in 0.65ms
Audited 5 packages in 0.11ms
Note: you may need to restart the kernel to use updated packages.


In [5]:
!grep -A 10 "scikit-learn" uv.lock

    { name = "scikit-learn" },
]

[package.metadata]
requires-dist = [{ name = "scikit-learn", specifier = "==1.6.1" }]

[[package]]
name = "numpy"
version = "2.3.4"
source = { registry = "https://pypi.org/simple" }
sdist = { url = "https://files.pythonhosted.org/packages/b5/f4/098d2270d52b41f1bd7db9fc288aaa0400cb48c2a3e2af6fa365d9720947/numpy-2.3.4.tar.gz", hash = "sha256:a7d018bfedb375a8d979ac758b120ba846a7fe764911a64465fd87b8729f4a6a", size = 20582187, upload-time = "2025-10-15T16:18:11.77Z" }
wheels = [
    { url = "https://files.pythonhosted.org/packages/96/7a/02420400b736f84317e759291b8edaeee9dc921f72b045475a9cbdb26b17/numpy-2.3.4-cp312-cp312-macosx_10_13_x86_64.whl", hash = "sha256:ef1b5a3e808bc40827b5fa2c8196151a4c5abe110e1726949d7abddfe5c7ae11", size = 20957727, upload-time = "2025-10-15T16:15:44.9Z" },
    { url = "https://files.pythonhosted.org/packages/18/90/a014805d627aa5750f6f0e878172afb6454552da929144b3c07fcae1bb13/numpy-2.3.4-cp312-cp312-macosx_11_0_arm64.whl", hash = "

In [6]:
import pickle

In [7]:
!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

--2025-10-27 13:46:18--  https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin [following]
--2025-10-27 13:46:18--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [application/octet-stream]
Saving to: ‘pipeline_v1.bin.3’

pipeline_

In [8]:
%%writefile predict.py

import pickle
with open("pipeline_v1.bin", "rb") as f_in:
    pipeline = pickle.load(f_in)


record = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

prediction = pipeline.predict_proba([record])[0, 1]

print("Predicted probability:", prediction)

Overwriting predict.py


In [9]:
!python predict.py

/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 1.7.0. Th

In [10]:
!pip install fastapi uvicorn


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [11]:
%%writefile app.py

import pickle
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

with open("pipeline_v1.bin", "rb") as f_in:
    pipeline = pickle.load(f_in)


class Client(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float

class PredictResponse(BaseModel):
    conversion_probability : float
    will_convert : bool

app = FastAPI(title = 'Prediction for Conversion')

@app.post("/predict", response_model = PredictResponse)
def predict(client: Client):
    record = client.dict()
    prob = pipeline.predict_proba([record])[0, 1]
    will_convert = prob >= 0.5
    
    return PredictResponse(
        conversion_probability = float(prob),
        will_convert = will_convert
    )

if __name__ == '__main__':
    uvicorn.run(app, host = '0.0.0.0', port = 3000)
        

Overwriting app.py


In [ ]:
!python app.py

/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 1.7.0. Th

In [1]:
%%writefile request.py
import requests

url = "http://localhost:3000/predict"  

client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client).json()
print(response)

Overwriting request.py


In [2]:
import threading
import uvicorn
from app import app  # make sure this matches your filename

def run_app():
    uvicorn.run(app, host="127.0.0.1", port=3000)

threading.Thread(target=run_app, daemon=True).start()

/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 1.7.0. Th

INFO:     127.0.0.1:51258 - "POST /predict HTTP/1.1" 200 OK


In [3]:
!python request.py

{'conversion_probability': 0.5340417283801275, 'will_convert': True}


In [1]:
%%writefile pyproject.toml
[project]
name = "homework"
version = "0.1.0"
description = "FastAPI service"
authors = [{ name = "Gao Qi Sheng" }]
dependencies = [
    "fastapi>=0.120.0",
    "uvicorn>=0.38.0",
    "scikit-learn == 1.6.1",
]

Overwriting pyproject.toml


In [2]:
%%writefile Dockerfile
FROM agrigorev/zoomcamp-model:2025

WORKDIR /app

COPY pyproject.toml .

RUN pip install uv && uv pip install --system -r <(uv pip compile pyproject.toml -q -o -)

COPY app.py .

EXPOSE 3000

CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "3000"]

Writing Dockerfile


In [3]:
!pwd

/workspaces/machine-learning-zoomcamp-homework/machine_learning_zoomcamp_homework_05


In [4]:
!dir

Dockerfile   app.py	 pipeline_v1.bin    pipeline_v1.bin.3  request.py
README.md    main.ipynb  pipeline_v1.bin.1  predict.py	       uv.lock
__pycache__  main.py	 pipeline_v1.bin.2  pyproject.toml
